# <center>Algorithmes classiques - Evaluation NSI - 1<sup>ère</sup></center>
## <center>Le bon mode de scrutin</center>

**Enoncé du problème** : six régions vont voter pour la présidence de l'union européenne. Cinq candidats se présentent à l'élection :   
- Le candidat français : François
- Le candidat espagnol : Esperanza
- Le candidat allemand : Ute 
- Le candidat italien : Marcello
- Le candidat anglais : John 


On connait les intentions de votes des six région et on souhaite étudier différents modes de scrutins afin de déterminer celui qui est le plus favorable à un candidat : 
- Election à un tour 
- Election à deux tours
- Election par élimination (last man standing)
- Système de Borda
- Système de Condorcet

On connait le nombre d'électeurs de chaque région et leurs intentions de vote pour les cinq candidats, par ordre de préférence : 

| | Région 1 (7,2 M) | Région 2 (4,8 M) | Région 3 (4M) | Région 4 (3,6) | Région 5 (1,6 M) | Région 6 (0,8)
| - | :-: | :-: | :-: | :-: | :-: | :-: |
|1|![FR](images/FR.png)|![ALL](images/ALL.png)|![ES](images/ES.png)|![IT](images/IT.png)|![UK](images/UK.png)|![UK](images/UK.png)|
|2|![IT](images/IT.png)|![UK](images/UK.png)|![ALL](images/ALL.png)|![ES](images/ES.png)|![ALL](images/ALL.png)|![ES](images/ES.png)|
|3|![UK](images/UK.png)|![IT](images/IT.png)|![UK](images/UK.png)|![UK](images/UK.png)|![IT](images/IT.png)|![IT](images/IT.png)|
|4|![ES](images/ES.png)|![ES](images/ES.png)|![IT](images/IT.png)|![ALL](images/ALL.png)|![ES](images/ES.png)|![ALL](images/ALL.png)|
|5|![ALL](images/ALL.png)|![FR](images/FR.png)|![FR](images/FR.png)|![FR](images/FR.png)|![FR](images/FR.png)|![FR](images/FR.png)|


## Simulation des différents modes de scrutin
### 1. Structure des données
Dans ce problème, on adopte les structures de données suivantes : 

In [1]:
# on crée un dictionnaire pour chacune des six régions
region1 = {'nb_electeurs': 7200000, 'candidats': ["Francois", "Marcello", "John", "Esperanza", "Ute"]}
region2 = {'nb_electeurs': 4800000, 'candidats': ["Ute", "John", "Marcello", "Esperanza", "Francois"]}
region3 = {'nb_electeurs': 4000000, 'candidats': ["Esperanza", "Ute", "John", "Marcello", "Francois"]}
region4 = {'nb_electeurs': 3600000, 'candidats': ["Marcello", "Esperanza", "John", "Ute", "Francois"]}
region5 = {'nb_electeurs': 1600000, 'candidats': ["John", "Ute", "Marcello", "Esperanza", "Francois"]}
region6 = {'nb_electeurs': 800000, 'candidats': ["John", "Esperanza", "Marcello", "Ute", "Francois"]}

# on regroupe les régions dans une liste
regions = [region1, region2, region3, region4, region5, region6]
print(regions)

[{'nb_electeurs': 7200000, 'candidats': ['Francois', 'Marcello', 'John', 'Esperanza', 'Ute']}, {'nb_electeurs': 4800000, 'candidats': ['Ute', 'John', 'Marcello', 'Esperanza', 'Francois']}, {'nb_electeurs': 4000000, 'candidats': ['Esperanza', 'Ute', 'John', 'Marcello', 'Francois']}, {'nb_electeurs': 3600000, 'candidats': ['Marcello', 'Esperanza', 'John', 'Ute', 'Francois']}, {'nb_electeurs': 1600000, 'candidats': ['John', 'Ute', 'Marcello', 'Esperanza', 'Francois']}, {'nb_electeurs': 800000, 'candidats': ['John', 'Esperanza', 'Marcello', 'Ute', 'Francois']}]


#### **Question 1**
A partir des structures de données ci-dessus, écrire l'instruction qui, à partir de ``regions``, permet : 
* d'afficher le nombre d'électeurs de la région ``region2`` 





* d'afficher le deuxième candidat de la dernière région ``region6`` (on pourra utiliser les index négatifs)





* d'afficher la sous-liste composée des deux premiers candidats de la première région ``region1`` 






### 2. Tri par insertion
Nous allons avoir besoin de trier des listes de dictionnaires selon l'ordre décroissant des valeurs d'une clé (correspondant aux nombres d'électeurs, de voix, de points...).
Voici un algorithme de tri par insertion qu'on va pouvoir utiliser :

In [2]:
def liste_triInsertionDictionnaire(liste ,cle) : 
    for i in range(1, len(liste)): #on commence le tri de liste[0:i] 
        j = i - 1 
        tmp = liste[i] #on stocke la valeur de liste[i] 
        while j > -1 and liste[j][cle] < tmp[cle]: 
            liste[j+1] = liste[j] #on decale les valeurs à droite 
            j -= 1 
            liste[j+1] = tmp # on place la valeur de l'ancien liste[i] 
    return liste 

#### **Question 2** 
Donner la complexité du tri par insertion dans le pire des cas, c'est à dire lorsque les données sont triées par ordre croissant.

#### **Question 3**
Décrire un invariant de boucle de la fonction ``liste_triInsertionDictionnaire``

### 3. Scrutin à un tour
Un candidat remporte le nombre de voix de la région où il est arrivé en tête. 

S'il arrive en tête dans plusieurs régions, les voix sont ajoutés. 

Celui qui remporte le plus de voix est vainqueur de l'élection à un tour. 

In [8]:
def election_un_tour(regions : list) -> list :
    """
    Renvoie la liste ordonnée des vainqueurs au premier tour. 
    Chaque vainqueur est un dictionnaire ayant pour clés 'candidat' (str) et 'voix' (int)
    Préconditions : liste de dictionnaires ayant pour clés 'candidats' (liste de caractères)
    et 'nb_electeurs' (int) 
    Postconditions : liste de dictionnaires 
    """    
    
    resultats = []
    # on récupère la liste des candidats à partir des intentions de vote de la première région
    # (tous les candidats sont représentés dans les intentions de vote de chaque région) 
    candidats = regions[0]['candidats']

    # resultats est une liste de dictionnaires qui contiennent le nom de chaque candidat et 
    # le nombre de voix qu'il obtient à l'issue d'un tour de scrutin. 
    # On initialise le nombre de voix de chaque candidat à 0     
    for candidat in candidats :
        resultats.append({'candidat' : candidat, 'voix' : 0}) 
        
    # pour chaque région 
    for region in regions :  
        # le favori est le candidat arrivé en tête des intentions
        favori = region['candidats'][0]                   
        
        # on ajoute les voix des électeurs de la région au favori  
        for candidat in resultats :
            if candidat['candidat'] == favori : 
                candidat['voix'] += region['nb_electeurs'] 
                
    return liste_triInsertionDictionnaire(resultats, 'voix')

In [4]:
print(election_un_tour(regions))

[{'candidat': 'Francois', 'voix': 7200000}, {'candidat': 'Ute', 'voix': 4800000}, {'candidat': 'Esperanza', 'voix': 4000000}, {'candidat': 'Marcello', 'voix': 3600000}, {'candidat': 'John', 'voix': 2400000}]


#### **Question 4**
Une petite question pour cet algo ?

### 4. Scrutin à deux tours
Les deux candidats en tête du premier tour sont qualifiés pour le 2<sup>nd</sup> tour. 

Au second tour, on reporte le nombre de voix des régions qui n'ont pas voté pour les qualifiés du premier tour sur celui le mieux placé dans les intentions de vote de la région. 

Le vainqueur est celui qui a obtenu le plus de voix à l'issue du deuxième tour.

In [16]:
def election_deux_tours(regions : list) -> list :
    """
    Renvoie la liste ordonnée des vainqueurs. 
    Chaque vainqueur est un dictionnaire ayant pour clés 'candidat' (str) et 'voix' (int)
    Préconditions : liste de dictionnaires ayant pour clés 'candidats' (liste de caractères)
    et 'nb_electeurs' (int) 
    Postconditions : liste de dictionnaires 
    """
    
    # on récupère les deux candidats en tête du premier tour
    vainqueurs_premier_tour = election_un_tour(regions)
    qualifie1 = vainqueurs_premier_tour[0]['candidat']
    qualifie2 = vainqueurs_premier_tour[1]['candidat']

    # pour le deuxième tour, on crée une liste pour comptabiliser 
    # les voix des deux favoris dans les régions où ils n'ont pas 
    # été choisis au premier tour
    vainqueurs_deuxieme_tour = {qualifie1 : 0, qualifie2 : 0}
    
    # on parcourt les régions
    for region in regions :
        if region['candidats'][0] == qualifie1 or region['candidats'][0] == qualifie2 :
            # Comment  
            continue
            
        # cette région n'a pas choisi les qualifiés au premier tour
        # on parcourt donc ses candidats jusqu'à rencontrer l'un des deux qualifiés 
        for i in range (1, len(region['candidats'])) :
            if region['candidats'][i] == qualifie1 or region['candidats'][i] == qualifie2 :
                vainqueurs_deuxieme_tour[region['candidats'][i]] += region['nb_electeurs']
                break # Comment

    # Mise en forme des résultats dans une liste de dictionnaires
    resultats_second_tour = []
    for candidat, nb_voix in vainqueurs_deuxieme_tour.items() :
        vainqueur = dict()
        vainqueur['candidat'] = candidat
        vainqueur['voix'] = nb_voix
        resultats_second_tour.append(vainqueur)
        
    return liste_triInsertionDictionnaire(resultats_second_tour, 'voix')       

In [6]:
print(election_deux_tours(regions))

[{'candidat': 'Ute', 'voix': 10000000}, {'candidat': 'Francois', 'voix': 0}]


#### **Question 5**
- La fonction ``election_deux_tours`` utilise les instructions ``continue`` et ``break``.
Que réalise ces deux instructions ? 

- Quels commentaires pourrait-on ajouter ligne 23 et 31 qui préciseraient le rôle de ces instructions dans le contexte de l'algorithme de la fonction ``election_deux_tours``? 

###  5. Scrutin par élimination
On réalise plusieurs  tours. A chaque tour, le candidat qui a le moins de voix est éliminé. 
Le candidat restant est le vainqueur. 

In [9]:
def election_par_elimination(regions : list) -> list:
    """
    Renvoie la liste ordonnée des vainqueurs. 
    Chaque vainqueur est un dictionnaire ayant pour clés 'candidat' (str) et 'voix' (int)
    Préconditions : liste de dictionnaires ayant pour clés 'candidats' (liste de caractères)
    et 'nb_electeurs' (int) 
    Postconditions : liste de dictionnaires 
    """
    
    # liste des candidat éliminés à chaque tour
    candidats_elimines = []
    nb_candidats = len(regions[0]['candidats'])
    nb_candidats_en_lice = nb_candidats
    
    regions_courantes = regions

    while nb_candidats_en_lice > 1 : 
        candidats_en_lice = election_un_tour(regions_courantes)
                               
        # on ajoute le candidat éliminé à la liste des candidats éliminés :
        candidats_elimines.append(candidats_en_lice[-1]["candidat"])
        nb_candidats_en_lice -= 1 # il y a donc un candidat en lice en moins
        
        # on reconstruit les régions avec, pour chacune d'elles, la liste des candidats moins ceux éliminés
        regions_courantes = []
        
        for region in regions :
            # nouvelle région avec le même nombre d'électeurs et la liste des candidats inititalisée à vide                    
            region_courante = {"nb_electeurs" : region["nb_electeurs"] , "candidats" : []} 
            
            for i in range(nb_candidats) :
                # on ajoute les candidats de la région non éliminés à la liste de candidats
                if region["candidats"][i] not in candidats_elimines :
                    region_courante["candidats"].append(region["candidats"][i]) 
                            
            # on ajoute la région actualisée à la liste des régions actualisées
            regions_courantes.append(region_courante)
              
    return candidats_en_lice

In [10]:
print(election_par_elimination(regions))

[{'candidat': 'Esperanza', 'voix': 14800000}, {'candidat': 'Francois', 'voix': 7200000}]


#### **Question 6** 
Justifier la terminaison de la fonction ``election_par_elimination`` 

### 6. Scrutin de Borda

In [11]:
def election_borda(regions : list) -> list :
    """
    Renvoie la liste ordonnée des vainqueurs. 
    Chaque vainqueur est un dictionnaire ayant pour clés 'candidat' (str) et 'voix' (int)
    Préconditions : liste de dictionnaires ayant pour clés 'candidats' (liste de caractères)
    et 'nb_electeurs' (int) 
    Postconditions : liste de dictionnaires 
    """
        
    # initialisation de la liste des candidats avec le nombre de voix à 0
    vainqueurs_borda = dict()
    nb_candidats = len(regions[0]['candidats'])
    for i in range(nb_candidats) : 
        vainqueurs_borda[regions[0]['candidats'][i]] = 0

    # pour chaque région, on calcule les points de chaque candidat coefficienté par son arrivée dans les intentions
    for region in regions : 
        coefficient = len(regions[0]['candidats'])
        for candidat in region['candidats'] : 
            vainqueurs_borda[candidat] += coefficient * region['nb_electeurs']
            coefficient -= 1 # on passe au candidat suivant, le coefficient baisse
            
    #on remet en forme les vainqueurs dans une liste de dictionnaires
    vainqueurs = []
    for candidat, nb_voix in vainqueurs_borda.items() :
        vainqueur = dict()
        vainqueur['candidat'] = candidat
        vainqueur['voix'] = nb_voix
        vainqueurs.append(vainqueur)
                          
    return liste_triInsertionDictionnaire(vainqueurs,'voix')

In [12]:
print(election_borda(regions))

[{'candidat': 'Marcello', 'voix': 76400000}, {'candidat': 'John', 'voix': 75600000}, {'candidat': 'Esperanza', 'voix': 64800000}, {'candidat': 'Ute', 'voix': 62400000}, {'candidat': 'Francois', 'voix': 50800000}]


#### **Question 7**
Décrire dans ses grandes lignes le principe du scrutin de Borda.


### 7. Scrutin de Condorcet
L'idée ici est de compter, pour chaque candidat, le nombre de duels gagnés contre les autres candidats. 

Le principe de comptage est le suivant : 
pour une région donnée, un candidat mieux placé dans les intentions de vote remporte contre les autre candidats  un nombre de points égal au nombre d'électeurs de la région. 

On itère sur toutes les régions en cumulant le nombre de points pour chaque des duels. 

Le (ou les) candidat(s) ayant remporté le plus de duels en nombre de points gagne(nt)

In [13]:
def election_condorcet(regions : list) -> list : 
    """
    Renvoie la liste ordonnée des vainqueurs. 
    Chaque vainqueur est un dictionnaire ayant pour clés 'candidat' (str) et 'voix' (int)
    Préconditions : liste de dictionnaires ayant pour clés 'candidats' (liste de caractères)
    et 'nb_electeurs' (int) 
    Postconditions : liste de dictionnaires 
    """
    
    # on initialise à 0 un tableau des duels à double entrée :
    # candidat1 en ligne versus candidat2 en colonne
    
    nb_candidats = len(regions[0]['candidats'])
    duels = []
    #une ligne par candidat
    for i in range(nb_candidats) : 
        ligne = []
        for j in range(nb_candidats) : 
             ligne.append(0)
        duels.append(ligne)
 
    # on crée une table d'index des candidats
    candidats = regions[0]['candidats']
    
    #pour chaque région
    for region in regions :
        #pour les duels des candidats de chaque région
        for candidat1 in region['candidats'] :
            index_candidat1 = region['candidats'].index(candidat1)
            # on ajoute le nombre d'electeurs à ses duels contre tous les candidats suivants  
            for candidat2 in region['candidats'][index_candidat1+1:] :
                duels[candidats.index(candidat1)][candidats.index(candidat2)] += region['nb_electeurs']

    # on identifie les duels gagnés 
    for i in range(len(candidats)) :
        for j in range(i+1, len(duels[i])) : 
            if duels[i][j] > duels[j][i] :
                duels[i][j] = 1 # duel gagné  
                duels[j][i] = 0 # duel perdu 
            else :
                duels[i][j] = 0 
                duels[j][i] = 1
                    
    # Comptage des duels gagnés et mise en forme des résultats dans une liste de disctionnaires 
    vainqueurs = []
    for candidat in candidats :
        vainqueur = dict()
        vainqueur['candidat'] = candidat
        vainqueur['duels'] = sum(duels[candidats.index(candidat)])
        vainqueurs.append(vainqueur)
                          
    return liste_triInsertionDictionnaire(vainqueurs,'duels')

In [14]:
print(election_condorcet(regions))

[{'candidat': 'John', 'duels': 4}, {'candidat': 'Marcello', 'duels': 3}, {'candidat': 'Esperanza', 'duels': 2}, {'candidat': 'Ute', 'duels': 1}, {'candidat': 'Francois', 'duels': 0}]


#### **Question 8***
Docstring ? 

###  8. Simulation de tous les modes de scrutins
Finalement, on simule tous les modes de scrutins sur les intentions de vote de nos six régions et on s'aperçoit que le vainqueur chanque à chaque fois ! 

In [15]:
print(election_un_tour(regions)[0]['candidat'],"gagne avec des élections à un tour")
print(election_deux_tours(regions)[0]['candidat'],"gagne avec des élections à deux tours")
print(election_par_elimination(regions)[0]['candidat'],"gagne avec des élections par élimination")
print(election_borda(regions)[0]['candidat'],"gagne avec un scrutin de Borda")
print(election_condorcet(regions)[0]['candidat'],"gagne avec un scrutin de Condorcet")

Francois gagne avec des élections à un tour
Ute gagne avec des élections à deux tours
Esperanza gagne avec des élections par élimination
Marcello gagne avec un scrutin de Borda
John gagne avec un scrutin de Condorcet
